In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Sujet HMM TP 1 

# Chaînes de Markov cachées (*Hidden Markov Models*, HMM)

On suppose que vous avez un ami qui habite dans un autre pays que vous. Chaque jous, cet ami joue au tennis ou bien bulle dans son canapé, en fonction de la météo.

Cependant, vous n'avez pas accès à la météo de ce pays, vous avez accès uniquement à l'information de ce qu'a fait votre ami (tennis ou canapé). 

On parle **d'observations** à propos des actions tennis/canapé, et **d'états cachés** pour la météo (3 états cachés ici).


Un **modèle de Markov caché** est un **modèle de séquence de symboles (mots, caractères, tags, phonèmes...) avec de l'information manquante associée à chaque symbole : son état**.

Notez que **la chaîne de Markov est définie sur les états cachés, pas sur les observations.**

Il s'agit d'un **modèle joint entre des symboles observables et leurs catégories latentes/cachées/inconnues**. 

Des hypothèses simplificatrices fortes sous-tendent les HMM : 



*   Dépendance de l'état présent à l'état précédent (1er ordre) uniquement
*   Stationnarité : les transitions entre états ne dépendent pas du temps
*   Indépendance statistique des observations entre elles




Dans ce TP, nous reprenons l'exemple du cours, illustré dans cette figure.


<img src="https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/cours2/exemple_HMM_meteo_correct.png" alt="chaine Markov" width="800"/>


Nous noterons les observations *canapé* et *tennis* $O_0$  et $O_1$, respectivement.

Arbitrairement nous attribuons un indice à chaque état : 


*   *Pluie* : 0
*   *Nuageux* : 1
*   *Soleil* : 2

On utilise les notations suivantes : 

*   $S_i$, avec $i=0,1,2$ pour désigner l'un de ces trois états.
*   $S^t$, avec $t$ entier positif pour désigner l'état dans lequel on se trouve à l'instant $t$.


Cet exemple est une chaîne de Markov dite d'ordre 1, au sens où la probabilité d'être dans l'état $S^t$ ne dépend que de l'état précédent $S^{t-1}$.


Elle serait d'ordre 2, si cette probabilité dépendait de $S^{t-1}$ et $S^{t-2}$.

**Question** : donner la matrice de transition $A=[a_{ij}]$ de cet exemple. 

Les éléments de $A$ sont les probabilités conditionnelles de passer de l'état $S_j$ à l'état $S_i$ : $a_{ij}=P(S_i|S_j)$.

In [4]:
A = torch.tensor([
    [0.5,0.4,0.0],
    [0.3,0.2,0.3],
    [0.2,0.4,0.7]])
A

tensor([[0.5000, 0.4000, 0.0000],
        [0.3000, 0.2000, 0.3000],
        [0.2000, 0.4000, 0.7000]])

En plus de la matrice de transition (entre les états **cachés**), nous introduisons une deuxième matrice, la matrice des probabilités d'obervation que nous notons $B=[B_{ij}]$. 

Les éléments de $B$ sont les probabilités conditionnelles "d'observer" *tennis* ou *canapé*, en fonction d'un état caché $S_j$ : 

$B_{ij}=P(O_i|S_j)$

Cette matrice s'appelle la **matrice d'émission**. Dans la figure, elle encode les probabilités des flêches rouges.

**Question** : donner la matrice de transition $B=[a_{ij}]$ de cet exemple. 


In [5]:
B =torch.tensor([
    [0.9,0.6,0.2],
    [0.1,0.4,0.8]
])
B

tensor([[0.9000, 0.6000, 0.2000],
        [0.1000, 0.4000, 0.8000]])

Le HMM de notre exemple est entièrement caractérisé par $\{A, B, \boldsymbol \pi\}$. Regroupons tout dans cette cellule, en calculant les valeurs de $\boldsymbol \pi$, avec la méthode des valeurs propres par exemple.

In [6]:
A = torch.tensor([
    [0.5,0.4,0.0],
    [0.3,0.2,0.3],
    [0.2,0.4,0.7]])

B = torch.tensor([
    [0.9,0.6,0.2],
    [0.1,0.4,0.8]
])

L, V = torch.linalg.eig(A)
pi0 = V[:,0]
pi0 = pi0/torch.sum(pi0)
pi0 = torch.real(pi0)

if torch.allclose(pi0, torch.tensor([0.2182, 0.2727, 0.5091]), atol=1e-04):
  print('OK !')
else:
  print('KO !')

# ces deux dict seront utiles par la suite
state2ind = {"Pluie": 0, "Nuageux": 1, "Soleil": 2}
ind2state = {0: "Pluie", 1: "Nuageux", 2: "Soleil"}

obs2ind = {"Canapé": 0, "Tennis": 1}
ind2obs = {0: "Canapé", 1: "Tennis"}

OK !


## **Question** : générer une séquence aléatoire d'observations de longueur 5

Pour cela, utiliser la fonction ```torch.multinomial()```

https://pytorch.org/docs/stable/generated/torch.multinomial.html

### Stratégie 1 : transition -> émission -> transition -> émission etc

In [44]:
N=5 # taille de la séquence que l'on veut
seq = [] # liste qui va contenir les observations générées en mots

# initialisation
current_state = torch.randint(3,(1,)).item()

# si on veut conserver la suite d'états (optionnel), créer cette liste
seq_states = [current_state]

for i in range(N):
  # émission
  current_obs = torch.multinomial(B[:,current_state],1).item()
  seq.append(current_obs)

  # transition
  current_state = torch.multinomial(A[:,current_state],1).item()
  seq_states.append(current_state)
  
current_obs = torch.multinomial(B[:,current_state],1).item()
seq.append(current_obs)
print(seq)

# optionnel
print(seq_states)

[1, 1, 0, 0, 1, 0]
[2, 2, 1, 0, 1, 1]


### Stratégie 2 (optionnel) : générer une séquence d'états cachés entière puis générer une séquence d'observations

In [54]:
N=5 # taille de la séquence que l'on veut

def gen_seq(N,A,B):
  seq = [] # liste qui va contenir les observations générées en mots

  # initialisation
  current_state = torch.randint(3,(1,)).item()

  # si on veut conserver la suite d'états (optionnel), créer cette liste
  seq_states = [current_state]

  for i in range(N):
    # transition
    current_state = torch.multinomial(A[:,current_state],1).item()
    seq_states.append(current_state)


  for i in range(N):
    # émission
    current_obs = torch.multinomial(B[:,seq_states[i]],1).item()
    seq.append(current_obs)

  current_obs = torch.multinomial(B[:,current_state],1).item()
  seq.append(current_obs) 
  return seq,seq_states

seq,seq_states = gen_seq(N,A,B)
print(seq)

# optionnel
print(seq_states)

[1, 1, 1, 1, 0, 1]
[2, 2, 1, 1, 1, 1]


## Probabilité d'une séquence d'observations données, lorsque la séquence d'états cachés associée est connue


\begin{equation}
P(O^0, \ldots, O^{T-1}, S^0, \ldots, S^{T-1}) = \boldsymbol \pi(S^0) \, P(O^0|S^0)\,Π_1^{T-1}\,\,P(S^{t}|S^{t-1})\,P(O^t|S^T)
\end{equation}

### Question 

Reprendre le code de génération de séquence ci-dessus et compléter pour calculer la probabilité de la séquence générée.

In [64]:
# seq,seq_states = gen_seq(N,A,B)
seq,seq_states = [0,1,1,0,0],[2,2,1,0,1]
P = pi0[seq_states[0]]*B[seq[0],seq_states[0]]

for i in range(1,N):
    previous_state = seq_states[i-1]
    current_state = seq_states[i]
    current_obs = seq[i]
    P *= A[current_state,previous_state]*B[current_obs,current_state]

print(P.item())

0.0004433734866324812


## Tutoriel HMM : le tutoriel "Rabiner" (1989)


*A Tutorial on Hidden Markov Models and Selected Applications in Speech Recognition* par Lawrence R. Rabiner, publié en 1989 dans Proc. of the IEEE.

Une version est disponible ici : https://www.cs.ubc.ca/~murphyk/Bayes/rabiner.pdf

Rabiner y décrit trois problèmes fondamentaux liés aux HMM :


1.   **Évaluation**. Étant donnée une séquence d'observations $O = O^0, ..., O^{T-1}$, et un modèle HMM $\lambda = \{\boldsymbol \pi, A, B\}$, comment peut-on calculer efficacement la probabilité $P(O|\lambda)$ de la séquence $O$, étant donné le modèle ?

2.   **Inférence ou décodage**. Étant donnée une séquence d'observations $O = O^0, ..., O^{T-1}$, et un modèle HMM $\lambda = \{\boldsymbol \pi, A, B\}$, comment retrouver la séquence d'états $S = S^0, ..., S^{T-1}$ optimale pour expliquer la séquence d'observations $O$ ?

3.   **Apprentissage**. comment ajuster les paramètres du modèle $\lambda = \{\boldsymbol \pi, A, B\}$ pour maximiser la vraisemblance (likelihood) de données d'entraînement, c'est-à-dire maximiser $P(O|\lambda)$.
 
Ces trois problèmes font appel à des algorithmes très astucieux : 

1.   **Évaluation**. Algorithme *forward* (il existe aussi la version *backward*, utile pour le problème 3)
2.   **Inférence ou décodage** : algorithme de *Viterbi* 
3.   **Apprentissage**. Deux situations possibles :

      a.   Apprentissage supervisé : on a les séquences $O$ et $S$ correspondantes. Alors on utilise l'apprentissage par maximum de vraisemblance, qui donne des formules analytiques pour calculer $\boldsymbol \pi, A, B$ à partir des fréquences d'occurrences des observations et des états.
      
      b.   Apprentissage non-supervisé : on a des séquences $O$ mais pas $S$. Algorithme de type *Expectation-Maximization* (EM) appelé la méthode *Baum-Welch* dans le tutoriel.


Les algorithmes *forward* et *Viterbi* sont des algorithmes de programmation dynamique, tout comme les algorithmes de distance de Levenshtein minimale, de recherche du plus court chemin de Dijkstra, etc. 


Dans ce TP, nous nous intéresserons seulement aux deux premiers problèmes de Rabiner, et coder les deux algorithmes *forward* et *Viterbi*. S'il reste du temps, il y a aussi en exercice le décodage *posterior*.



## Problème 1 : **Évaluation**, coder l'algorithme *forward*

Étant donnée une séquence d'observations $O = O^0, ..., O^{T-1}$, et un modèle HMM $\lambda = \{\boldsymbol \pi, A, B\}$, comment peut-on calculer efficacement la probabilité $P(O|\lambda)$ de la séquence $O$, appelée vraisemblance ou *likelihood*, étant donné le modèle ?

### Approche 1 : force brute (pas de code à écrire)

Force brute : considérer toutes les séquences d'états valides, qui pourraient générer $O$ et sommer les probabilités correspondantes.

Considérons la séquence *Canapé -> Tennis*, lister tous les cas à envisager et les probabilités associées.

Il y a NxN cas soit ici 9 cas. Si on note U pour Pluie, N pour Nuageux et S pour Soleil, on doit calculer :

<!-- \begin{eqnarray}
P(01, UU) &=& \boldsymbol \pi(U)P(0|U)& P(U|U)P(1|U)\\
P(01, UN) &=& \_\_\_\_\_\_\_\_\_\_ & P(N|U)P(1|N)\\
P(01, US) &=& \_\_\_\_\_\_\_\_\_\_ & P(S|U)\,\,P(1|S)\\
P(01, NU) &=& \boldsymbol \pi(N)P(0|N)& P(U|N)P(1|U)\\
P(01, NN) &=& \_\_\_\_\_\_\_\_\_\_ & P(N|N)P(1|N)\\
P(01, NS) &=& \_\_\_\_\_\_\_\_\_\_ & P(S|N)\,\,P(1|S)\\
P(01, SU) &=& \boldsymbol \pi(S)P(0|S)& P(U|S)P(1|U)\\
P(01, SN) &=& \_\_\_\_\_\_\_\_\_\_ & P(N|S)P(1|N)\\
P(01, SS) &=& \_\_\_\_\_\_\_\_\_\_ & P(S|S)\,\,P(1|S)\\
\end{eqnarray} -->


\begin{eqnarray}
P(01, UU) &=& \boldsymbol \pi(U)P(0|U)& P(U|U)P(1|U)\\
P(01, NU) &=& \boldsymbol \pi(N)P(0|N) & P(U|N)P(1|U)\\
P(01, SU) &=& \boldsymbol \pi(S)P(0|S) & P(U|S)\,\,P(1|U)\\
P(01, UN) &=& \boldsymbol \pi(U)P(0|U)& P(N|U)P(1|N)\\
P(01, NN) &=& \boldsymbol \pi(N)P(0|N) & P(N|N)P(1|N)\\
P(01, SN) &=& \boldsymbol \pi(S)P(0|S) & P(N|S)\,\,P(1|N)\\
P(01, US) &=& \boldsymbol \pi(U)P(0|U)& P(S|U)P(1|S)\\
P(01, NS) &=& \boldsymbol \pi(N)P(0|N) & P(S|N)P(1|S)\\
P(01, SS) &=& \boldsymbol \pi(S)P(0|S) & P(S|S)\,\,P(1|S)\\
\end{eqnarray}

On voit des redondances. Posons : 

\begin{eqnarray}
\alpha_U &=& \boldsymbol \pi(U)P(0|U)\\
\alpha_N &=& \boldsymbol \pi(N)P(0|N)\\
\alpha_S &=& \boldsymbol \pi(S)P(0|S)\\
\end{eqnarray}

On a :

\begin{eqnarray}
P(01, UU) &=& \alpha_U & P(U|U)P(1|U)\\
P(01, NU) &=& \alpha_N & P(U|N)P(1|U)\\
P(01, SU) &=& \alpha_S & P(U|S)\,\,P(1|U)\\
P(01, UN) &=& \alpha_U & P(N|U)P(1|N)\\
P(01, NN) &=& \alpha_N & P(N|N)P(1|N)\\
P(01, SN) &=& \alpha_S & P(N|S)\,\,P(1|N)\\
P(01, US) &=& \alpha_U & P(S|U)P(1|S)\\
P(01, NS) &=& \alpha_N & P(S|N)P(1|S)\\
P(01, SS) &=& \alpha_S & P(S|S)\,\,P(1|S)\\
\end{eqnarray}



Faisons les calculs

In [65]:
alpha_canape = pi0 * B[0]
alpha_U = alpha_canape[0]
alpha_N = alpha_canape[1]
alpha_S = alpha_canape[2]
print(alpha_canape)

p1 = alpha_U * A[0,0] * B[1, 0]
p2 = alpha_N * A[0,1] * B[1, 0]
p3 = alpha_S * A[0,2] * B[1, 0]

p4 = alpha_U * A[1,0] * B[1, 1]
p5 = alpha_N * A[1,1] * B[1, 1]
p6 = alpha_S * A[1,2] * B[1, 1]

p7 = alpha_U * A[2,0] * B[1, 2]
p8 = alpha_N * A[2,1] * B[1, 2]
p9 = alpha_S * A[2,2] * B[1, 2]

p = p1 + p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9
print("Probabilité d'observer Canapé -> Tennis : {:.3f}".format(p))

tensor([0.1964, 0.1636, 0.1018])
Probabilité d'observer Canapé -> Tennis : 0.206


Bilan


Pour une séquence de longueur $T$ et $N$ états cachés, le nombre de multiplications à faire pour calculer cette probabilité est de l'ordre de $2TN^T$.

Autrement dit, la complexité est $O(N^T)$, soit exponentielle par rapport au nombre d'états cachés.

Il faut donc faire autrement...

### Approche 2 : l'algorithme forward (à vous de jouer)

Si l'on énumère les séquences d'états possibles pour générer une séquence d'observation, on s'aperçoit qu'il y a beaucoup de calculs en commun, et donc redondants.

L'algorithme *forward* est un algorithme de programmation dynamique.

Il utilise une variable intermédiaire appelée $\alpha$ :

$\alpha^t(S_i)$ est la probabilité jointe de la séquence partielle d'observations $O^0, \ldots, O^t$ tout en étant dans l'état $S_i$ à l'instant $t$ : 

\begin{equation}
\alpha^t(S_i) = P(O^0, \ldots, O^t, S^t=S_i)
\end{equation}

L'algorithme *forward* est le suivant : 



*   Initialisation

   *   Pour tous les états $i=0\ldots N-1$, poser :
      \begin{equation}
        \alpha^0(S_i) = \boldsymbol \pi_i P(O^0|S_i)
      \end{equation}

      À noter que $P(O^0|S_i)$ est un élément de la matrice d'émission $B$. 

*   Récurrence/induction

    *   Pour les itérations suivantes, d'indices $t=1\ldots T-1 $, et pour les $N$ états $i=0\ldots N-1$, calculer :

\begin{equation}
    \alpha^t(S_i) = \sum_{j=0}^{N-1} \alpha^{t-1}(S_j)\,P(S_i|S_j)\,P(O^t|S_i)\\ 
  = P(O^t|S_i) \sum_{j=0}^{N-1} \alpha^{t-1}(S_j)\,P(S_i|S_j)
\end{equation}

*   Terminaison

    *   On obtient finalement la probabilité de la séquence complète : 

        \begin{equation}
          P(O) = \sum_{j=0}^{N-1} \alpha^{T-1}(S_j) 
        \end{equation}





<img src="https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/cours2/forward.png" alt="forward" width="300"/>





#### Coder une première version "naïve", avec des boucles *for* imbriquées pour l'étape d'induction.

Votre fonction prend en entrée une séquence d'observations et les paramètres du modèle, et retourne la probabilité de la séquence.

In [71]:
def forward_naif(seq, A, B, pi0):
    # seq : sequence d'observations, liste d'entiers
    # A : matrice de transition N x N où N nb d'états cahcés
    # B : matrice d'émission O x N où O nb d'observations possibles différentes
    # pi0: vecteur distribution stationnaire des N états cachés

    T = len(seq) # longueur de la seq 
    N = A.size(0) # nb d'états cachés

    # initialisation
    alpha = torch.zeros(N,) 
    for i in range(N):
        alpha[i] = pi0[i]*B[seq[0],i]
    
    # induction
    for t in range(1,T):  
        alpha_prev = torch.clone(alpha)
        for i in range(N):
            sum = 0
            for j in range(N):
                sum+= alpha_prev[j]*A[i,j]
            alpha[i] = B[seq[t],i]*sum
    # terminaison
    res = 0
    for j in range(N):
        res+=alpha[j]
    return res

Tester votre fonction :

In [72]:
seq_obs = [0,1]
seq_obs_mots = [ind2obs[i] for i in seq_obs]

p = forward_naif(seq_obs, A, B, pi0)
print(" -> ".join(seq_obs_mots))
print("Probabilité : {:.3f}".format(p))

if torch.isclose(p, torch.tensor(0.206), atol=1e-03):
  print('OK !')
else:
  print('KO !')

Canapé -> Tennis
Probabilité : 0.206
OK !


#### Amélioration

Dans l'étape d'induction, ne garder que la boucle sur les éléments de la séquence et remplacer les autres boucles imbriquées par un produit matriciel.

In [87]:
def forward(seq, A, B, pi0):
    # seq : sequence d'observations, liste d'entiers
    # A : matrice de transition N x N où N nb d'états cahcés
    # B : matrice d'émission O x N où O nb d'observations possibles différentes
    # pi0: vecteur distribution stationnaire des N états cachés

    T = len(seq) # longueur de la seq 
    N = A.size(0) # nb d'états cachés

    # initialisation
    alpha = torch.zeros(N,) 
    for i in range(N):
        alpha[i] = pi0[i]*B[seq[0],i]

    # induction
    for t in range(1,T):
        alpha = B[seq[t],:] * torch.matmul(A,alpha)
        
    # terminaison
    res = 0
    for j in range(N):
        res+=alpha[j]
    return res



Tester votre fonction

In [88]:
seq_obs = [0,1]
seq_obs_mots = [ind2obs[i] for i in seq_obs]

p = forward(seq_obs, A, B, pi0)
print(" -> ".join(seq_obs_mots))
print("Probabilité : {:.3f}".format(p))

if torch.isclose(p, torch.tensor(0.206), atol=1e-03):
  print('OK !')
else:
  print('KO !')

Canapé -> Tennis
Probabilité : 0.206
OK !


Tester votre fonction sur une séquence plus longue

In [89]:
seq_obs = [0,1,1,1,0,0]

seq_obs_mots = [ind2obs[i] for i in seq_obs]

p = forward(seq_obs, A, B, pi0)
print(" -> ".join(seq_obs_mots))
print("Probabilité : {:.3f}".format(p))

if torch.isclose(p, torch.tensor(0.014), atol=1e-03):
  print('OK !')
else:
  print('KO !')


Canapé -> Tennis -> Tennis -> Tennis -> Canapé -> Canapé
Probabilité : 0.014
OK !


**Question** : pourquoi est-ce que la probabilité de cette séquence est plus petite que celle de la séquence précédente ?

Plus une séquence est longue, plus elle est spécifique et donc moins elle a de chances de survenir.

La complexité de cet algorithme est $O(N^2T)$, bien mieux que $O(N^T)$ !!

## Problème 2 : **Inférence ou décodage**

### Exemple tennis ou canap'?

Nous souhaitons écrire un algorithme qui remplisse le tableau suivant avec les scores de chaque case, ainsi que, dans une deuxième temps, l'état d'où l'on vient à l'instant d'avant, pour obtenir la séquence d'états optimale.

En effet, ce qui nous intéresse dans le décodage, c'est d'obtenir la séquence de prédictions des états cachés.

<img src="https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/cours2/viterbi_meteo.png" alt="viterbi" width="800"/>

### Exemple de cas d'usage : en traitement de données textuelles (*NLP* pour *Natural Language Processing*), trouver les tags Part-Of-Speech des mots d'une phrase (*POS tagging*) : 

<img src="https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/cours2/problem2_exemple_POS_tagging.png" alt="viterbi" width="800"/>

4000 séquences d'états possibles sur ce petit exemple !

(Image de Noah Smith)

### Formulation

Étant donnée une séquence d'observations $O = O^0, ..., O^{T-1}$, et un modèle HMM $\lambda = \{\boldsymbol \pi, A, B\}$, comment retrouver la séquence d'états $S = S^0, ..., S^{T-1}$ optimale pour expliquer la séquence d'observations $O$ ?


Nous pourrions vouloir simplement choisir l'état le plus vraisemblable à chaque temps $t$, indépendamment des états précédents ou suivants : selon $P(S^t=S_i|O)$. Ce serait très efficace, mais cela peut donner des séquences non-valides, si par exemple il y a des probabilités de transition nulles dans le modèle.

L'algorithme de référence est l'algorithme Viterbi, qui est un algorithme de programmation dynamique.



### Viterbi (scores seulement)

Nous allons calculer à chaque temps $t$, le meilleur score (plus grande vraisemblance) d'un unique meilleur chemin, qui rend compte des $t$ premières observations, et qui aboutit à l'état $S_i$ au temps $t$. Dans le tutoriel Rabiner, ce score est noté par la lettre grecque delta : 

\begin{equation}
\delta^t(S_i) = \max_{S^0, S^1, \ldots, S^{t-1}} P(S^0, O^0, S^1, O^1, \ldots, S^t=S_i, O^t)
\end{equation}


*   Initialisation

   *   Pour tous les états $i=0\ldots N-1$, poser :
      \begin{equation}
        \delta^0(S_i) = \boldsymbol \pi_i P(O^0|S_i)
      \end{equation}

      À noter que $P(O^0|S_i)$ est un élément de la matrice d'émission $B$. 

*   Récurrence/induction

    *   Pour les itérations suivantes, d'indices $t=1\ldots T-1 $, et pour les $N$ états $i=0\ldots N-1$, calculer :

      \begin{align*}
          \delta^t(S_i) &=& \max_{j=0\ldots N-1} \delta^{t-1}(S_j)\,P(S_i|S_j)\,P(O^t|S_i)\\ 
           &=& P(O^t|S_i) \max_{j=0\ldots N-1} \delta^{t-1}(S_j)\,P(S_i|S_j)
      \end{align*}

*   Terminaison

    *   On obtient finalement le score de la séquence la plus vraisemblable des états : 

        \begin{equation}
          \text{score}^{T-1} = P(S^*|O) = \max_{j=0\ldots N-1} \delta^{T-1}(S_j) 
        \end{equation}



#### Exemple POS tagging

<img src="https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/cours2/viterbi_example_POS_scores.png" alt="viterbi POs example" width="800"/>


(Image de Noah Smith)

#### Code **Exercice** 

Votre fonction prend en entrée une séquence d'observations et les paramètres du modèle, et retourne le score de la séquence.

In [ ]:
def viterbi_score(seq, A, B, pi0):
    # seq : sequence d'observations, liste d'entiers
    # A : matrice de transition N x N où N nb d'états cahcés
    # B : matrice d'émission O x N où O nb d'observations possibles différentes
    # pi0: vecteur distribution stationnaire des N états cachés

    T = len(seq) # longueur de la seq 
    N = A.size(0) # nb d'états cachés

    # initialisation
    delta = torch.zeros(N,) 
    for i in range(N):
        delta[i] = pi0[i]*B[seq[0],i]

    # induction
    for t in range(1,T):  
        delta_prev = torch.clone(delta)
        for i in range(N):
            sum = 0
            for j in range(N):
                sum+= alpha_prev[j]*A[i,j]
            alpha[i] = B[seq[t],i]*sum
    # terminaison
    res = 0
    for j in range(N):
        res+=alpha[j]
    
    return res

In [ ]:
seq_obs = [0,1]
seq_obs_mots = [ind2obs[i] for i in seq_obs]

score = viterbi_score(seq_obs, A, B, pi0)
print(" -> ".join(seq_obs_mots))
print("Score Viterbi : {:.3f}".format(score))

if torch.isclose(score, torch.tensor(0.057), atol=1e-03):
  print('OK !')
else:
  print('KO !')

#### **Question : est-ce qu'on n'a pas fait la même chose déjà avec l'algorithme *forward* ?**

<img src="https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/cours2/diff_forward_viterbi.png" alt="chaine Markov" width="500"/>


### Viterbi (scores et séquence d'états)

Viterbi nous donne le score du meilleur chemin, mais comment avoir les prédictions (la séquence d'états cachés) qui correspondent ?

Lors du calcul des scores par induction, à chaque temps $t$, il faut conserver l'identité de l'état dont le score est maximal. 

Ces états seront notés $\text{bp}$ pour *backpointers* en anglais.

Voici l'algo complété.

*   Initialisation

   *   Pour tous les états $i=0\ldots N-1$, poser :
      \begin{equation}
        \delta^0(S_i) = \boldsymbol \pi_i P(O^0|S_i)
      \end{equation}

      À noter que $P(O^0|S_i)$ est un élément de la matrice d'émission $B$. 

*   Récurrence/induction

    *   Pour les itérations suivantes, d'indices $t=1\ldots T-1 $, et pour les $N$ états $i=0\ldots N-1$, calculer :

      \begin{eqnarray}
          \delta^t(S_i) &=& \max_{j=0\ldots N-1} \delta^{t-1}(S_j)\,P(S_i|S_j)\,P(O^t|S_i)\\ 
           &=& P(O^t|S_i) \max_{j=0\ldots N-1} \delta^{t-1}(S_j)\,P(S_i|S_j)
      \end{eqnarray}
    
    * Déterminer quel est l'état au temps précédent qui a maximisé le score :   
    \begin{equation}
        \text{bp}^t(S_i) = \text{arg max}_{j=0\ldots N-1} \delta^{t-1}(S_j)\,P(S_i|S_j)
    \end{equation}

*   Terminaison

    *   On obtient finalement la probabilité de la séquence complète : 

        \begin{equation}
          P^*(O) = \max_{j=0\ldots N-1} \delta^{T-1}(S_j) 
        \end{equation}
    
    *   Et l'état optimal : 
    \begin{equation}
        \text{bp}^{T-1}(S_i) = \text{arg max}_{j=0\ldots N-1} \delta^{T-2}(S_j)\,P(S_i|S_j)
    \end{equation}


Puis pour récupérer le meilleur chemin d'états (*Backtracing*) :

   *   On choisit l'état appelé $S^{T-1}_*$ qui maximise $\delta^{T-1}$.
   *   Puis à l'instant $t$ précédent prendre : $S^{T-2} = \text{bp}^{T-1}(S^{T-1}_*)$.
   *   De manière générale, $S^{t-1} = \text{bp}^{t}(S^{t}_*)$.

   

#### Exemple POS tagging


<img src="https://www.irit.fr/~Thomas.Pellegrini/ens/M2ML2/cours2/viterbi_example_POS.png" alt="viterbi POs example" width="800"/>


(Image de Noah Smith)

#### Code **exercice**

Pour le backtracing, vous utiliserez la fonction ```torch.argmax()``` : 

https://pytorch.org/docs/stable/generated/torch.argmax.html

In [ ]:
def viterbi_complet(seq, A, B, pi0):
    # seq : sequence d'observations, liste d'entiers
    # A : matrice de transition N x N où N nb d'états cahcés
    # B : matrice d'émission O x N où O nb d'observations possibles différentes
    # pi0: vecteur distribution stationnaire des N états cachés

    T = len(seq) # longueur de la seq 
    N = A.size(0) # nb d'états cachés

    # initialisation
    ??
    
    # on ajoute ce tenseur pour stocker les meilleurs etats au long du scoring (backpointers)
    bp = torch.zeros(T,N, dtype=int)

    # induction
    ??

    # terminaison scoring
    res = ??

    # print(delta)
    # print(bp)

    # backtracing
    viterbi_seq = [??.item()]
    for t in range(??):
        best_state = ??
        viterbi_seq.append(??.item())

    # on remet viterbi_seq dans l'ordre t=0...T-1 :
    viterbi_seq = ??

    return res, viterbi_seq

In [ ]:
# seq_obs = [0,1,1,1,0,0]
seq_obs = [0,1]
seq_obs_mots = [ind2obs[i] for i in seq_obs]

score, seq_etats = viterbi_complet(seq_obs, A, B, pi0)
print(" -> ".join(seq_obs_mots))
print("Score : {:.5f}".format(score))
seq_etats_mots = [ind2state[i] for i in seq_etats]
print("Séquence d'états Viterbi : ", " -> ".join(seq_etats_mots))

if seq_etats == [0,2]:
  print('OK !')
else:
  print('KO !')
